# Kaggle 참조 데이터셋 로더

> **Purpose:** Kaggle 공개 데이터셋을 로드하여 합성 데이터 생성의 참조 및 검증에 활용합니다.
>
> **참조 데이터셋:**
> 1. Synthetic Healthcare Dataset: https://www.kaggle.com/datasets/divyabhavana/synthetic-healthcare-dataset
> 2. Healthcare Dataset: https://www.kaggle.com/datasets/prasad22/healthcare-dataset
> 3. RSNA STR Pulmonary Embolism Detection: https://www.kaggle.com/c/rsna-str-pulmonary-embolism-detection/data
> 4. SIIM-COVID19 Detection: https://www.kaggle.com/c/siim-covid19-detection/data

## 1. 라이브러리 및 설정

In [ ]:

from pyspark.sql import functions as F
from pyspark.sql.types import *

# ADLS Gen2 마운트 경로 (Azure Key Vault에서 연결 정보 가져오기)
ADLS_BASE_PATH = "/mnt/adls-gen2"
KAGGLE_DATA_PATH = f"{ADLS_BASE_PATH}/kaggle-reference"

## 2. Synthetic Healthcare Dataset 로드

합성 의료 데이터의 기본 구조 참조용

In [ ]:

# 스키마 정의
synthetic_healthcare_schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Gender", StringType(), True),
    StructField("Blood Type", StringType(), True),
    StructField("Medical Condition", StringType(), True),
    StructField("Date of Admission", StringType(), True),
    StructField("Doctor", StringType(), True),
    StructField("Hospital", StringType(), True),
    StructField("Insurance Provider", StringType(), True),
    StructField("Billing Amount", DoubleType(), True),
    StructField("Room Number", IntegerType(), True),
    StructField("Admission Type", StringType(), True),
    StructField("Discharge Date", StringType(), True),
    StructField("Medication", StringType(), True),
    StructField("Test Results", StringType(), True),
])

# CSV 로드 (파일이 업로드된 경우)
# df_synthetic = spark.read.csv(
#     f"{KAGGLE_DATA_PATH}/synthetic_healthcare_dataset.csv",
#     header=True,
#     schema=synthetic_healthcare_schema
# )

print("✅ Synthetic Healthcare Dataset 스키마 정의 완료")
print("   → 파일 업로드 후 주석 해제하여 로드")

## 3. Healthcare Dataset 로드 (HL7 CDA 참조)

임상 진료 기록 표준(HL7 CDA) 형식의 참조 데이터

In [ ]:

healthcare_schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Gender", StringType(), True),
    StructField("Blood Type", StringType(), True),
    StructField("Medical Condition", StringType(), True),
    StructField("Date of Admission", DateType(), True),
    StructField("Doctor", StringType(), True),
    StructField("Hospital", StringType(), True),
    StructField("Insurance Provider", StringType(), True),
    StructField("Billing Amount", DoubleType(), True),
    StructField("Room Number", IntegerType(), True),
    StructField("Admission Type", StringType(), True),
    StructField("Discharge Date", DateType(), True),
    StructField("Medication", StringType(), True),
    StructField("Test Results", StringType(), True),
])

# df_healthcare = spark.read.csv(
#     f"{KAGGLE_DATA_PATH}/healthcare_dataset.csv",
#     header=True,
#     schema=healthcare_schema
# )

print("✅ Healthcare Dataset 스키마 정의 완료")

## 4. DICOM 의료 영상 메타데이터 참조

RSNA 및 SIIM-COVID19 데이터셋의 DICOM 메타데이터 구조

In [ ]:

# DICOM 메타데이터 스키마 (주요 필드)
dicom_metadata_schema = StructType([
    StructField("StudyInstanceUID", StringType(), True),
    StructField("SeriesInstanceUID", StringType(), True),
    StructField("SOPInstanceUID", StringType(), True),
    StructField("PatientID", StringType(), True),
    StructField("Modality", StringType(), True),           # CT, MRI, CR(X-ray)
    StructField("BodyPartExamined", StringType(), True),
    StructField("StudyDate", StringType(), True),
    StructField("StudyDescription", StringType(), True),
    StructField("ImagePositionPatient", StringType(), True),
    StructField("PixelSpacing", StringType(), True),
    StructField("WindowCenter", DoubleType(), True),
    StructField("WindowWidth", DoubleType(), True),
    StructField("Rows", IntegerType(), True),
    StructField("Columns", IntegerType(), True),
])

# RSNA Pulmonary Embolism 진단 라벨 스키마
rsna_label_schema = StructType([
    StructField("StudyInstanceUID", StringType(), True),
    StructField("negative_exam_for_pe", IntegerType(), True),
    StructField("indeterminate", IntegerType(), True),
    StructField("chronic_pe", IntegerType(), True),
    StructField("acute_and_chronic_pe", IntegerType(), True),
    StructField("central_pe", IntegerType(), True),
    StructField("leftsided_pe", IntegerType(), True),
    StructField("rightsided_pe", IntegerType(), True),
    StructField("rv_lv_ratio_gte_1", IntegerType(), True),
    StructField("rv_lv_ratio_lt_1", IntegerType(), True),
])

print("✅ DICOM 메타데이터 및 RSNA 라벨 스키마 정의 완료")

## 5. 데이터 검증 유틸리티

In [ ]:

def validate_dataset_statistics(df, dataset_name):
    """데이터셋의 기본 통계량을 출력하여 품질을 검증합니다."""
    print(f"\n📊 {dataset_name} 통계 요약:")
    print(f"   총 레코드 수: {df.count():,}")
    print(f"   컬럼 수: {len(df.columns)}")
    print(f"   Null 비율:")
    
    for col_name in df.columns:
        null_count = df.filter(F.col(col_name).isNull()).count()
        total = df.count()
        null_pct = null_count / total * 100 if total > 0 else 0
        if null_pct > 0:
            print(f"     - {col_name}: {null_pct:.1f}%")
    
    print(f"\n   수치형 요약:")
    df.describe().show()

print("✅ 데이터 검증 유틸리티 준비 완료")
print("→ 데이터 로드 후 validate_dataset_statistics(df, 'DatasetName') 호출")